**Lab 4, Task 1  Sverdlyk Ivan, FB-51mp.**
В даній частині лабораторної роботи реалізовано машинний переклад з англійської на українську мову. Використано архітектуру Seq2Seq на базі RNN (GRU), з механізмом уваги.

In [1]:
# Вимушений був скористатись Colab, адже навчання відбувається дуже довго на CPU, а хотілось би побачити результат.
# Розпочав з підключення всього необхідного
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import unicodedata
import string
import re
import random
import time
import math
import os
# (GPU або CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Пристрій для навчання: {device}")

Пристрій для навчання: cuda


In [2]:
# Токени
SOS_token = 0  # Start Of Sentence (Початок)
EOS_token = 1  # End Of Sentence (Кінець)
UNK_token = 2  # Unknown (Невідоме слово)
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        # Ініціалізація з трьома токенами
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
        self.n_words = 3
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Очищення та нормалізація тексту
def normalizeString(s):
    # Нижній регістр, видалення пробілів
    s = s.lower().strip()
    # Відокремлюємо пунктуацію від слів
    s = re.sub(r"([.!?])", r" \1", s)
    # Залишаємо тільки літери (латиниця + кирилиця) та основні знаки
    s = re.sub(r"[^a-zA-Zа-яА-ЯіїєґІЇЄҐ.!?]+", r" ", s)
    return s

In [4]:
# Завантаження та фільтрація даних
MAX_LENGTH = 15  # Максимальна довжина
def readLangs(lang1, lang2, reverse=False):
    print("ukr.txt")
    if not os.path.exists('ukr.txt'):
        print("Помилка")
        return None, None, None
    lines = open('ukr.txt', encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
           len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    if not pairs: return None, None, None

    print(f"Прочитано {len(pairs)} пар речень")
    pairs = filterPairs(pairs)
    print(f"Відфільтровано до {len(pairs)} пар речень")
    print("Створення словників.")

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print(f"Слів у {input_lang.name}: {input_lang.n_words}")
    print(f"Слів у {output_lang.name}: {output_lang.n_words}")
    return input_lang, output_lang, pairs

In [5]:
# Модель енкодера
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [6]:
# Модель декодера, також вирішив використати Attention
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        # Attention
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        # Розрахунок коефіцієнтів уваги
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        # Застосування уваги до виходів енкодера
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
# Переведення у тензори для навчання
def indexesFromSentence(lang, sentence):
    indexes = []
    for word in sentence.split(' '):
        if word in lang.word2index:
            indexes.append(lang.word2index[word])
        else:
            indexes.append(UNK_token)
    return indexes

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [8]:
# Тренування для однієї пари
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    # Прохід через Encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    # Прохід через Decoder
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    # Teacher Forcing, 50% шанс використати правильну відповідь як підказку
    use_teacher_forcing = True if random.random() < 0.5 else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
# Допоміжні функції для логування
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [10]:
# Основний цикл навчання
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.001):
    start = time.time()
    print_loss_total = 0
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()
    print(f"Початок тренування: {n_iters} ітерацій.")
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [11]:
# Тестування роботи моделі.
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                word = output_lang.index2word.get(topi.item(), "<UNK>")
                decoded_words.append(word)

            decoder_input = topi.squeeze().detach()

        return decoded_words

def evaluateRandomly(encoder, decoder, n=5):
    print("\nТестова перевірка.")
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

def save_models(encoder, decoder):
    torch.save(encoder.state_dict(), 'encoder.pth')
    torch.save(decoder.state_dict(), 'decoder.pth')

In [12]:
# Підготовка даних
input_lang, output_lang, pairs = prepareData('eng', 'ukr', reverse=False)
if pairs:
    # Створення моделі
    hidden_size = 256
    encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
    trainIters(encoder1, attn_decoder1, n_iters=75000, print_every=5000, learning_rate=0.001)
    # Перевірка результатів
    evaluateRandomly(encoder1, attn_decoder1)
    # Збереження
    save_models(encoder1, attn_decoder1)

ukr.txt
Прочитано 160049 пар речень
Відфільтровано до 159274 пар речень
Створення словників.
Слів у eng: 10170
Слів у ukr: 30884
Початок тренування: 75000 ітерацій.
2m 21s (- 32m 59s) (5000 6%) 4.7644
4m 36s (- 29m 57s) (10000 13%) 4.2025
6m 50s (- 27m 22s) (15000 20%) 3.9907
9m 4s (- 24m 57s) (20000 26%) 3.9377
11m 18s (- 22m 37s) (25000 33%) 3.9086
13m 32s (- 20m 18s) (30000 40%) 3.9283
15m 47s (- 18m 2s) (35000 46%) 3.8698
18m 1s (- 15m 46s) (40000 53%) 3.9053
20m 15s (- 13m 30s) (45000 60%) 3.8998
22m 29s (- 11m 14s) (50000 66%) 3.8740
24m 43s (- 8m 59s) (55000 73%) 3.8564
26m 57s (- 6m 44s) (60000 80%) 3.8744
29m 12s (- 4m 29s) (65000 86%) 3.8615
31m 27s (- 2m 14s) (70000 93%) 3.7702
33m 41s (- 0m 0s) (75000 100%) 3.8106

Тестова перевірка.
> i relied on tom .
= я довірилася тому .
< я тома тома . <EOS>

> tom thought mary could win .
= том думав що мері може виграти .
< том думав що мері мері . <EOS>

> any student can answer that question .
= будь який студент може дати відповід